# Google Colab Setup

In [ ]:
!git clone https://github.com/haidarihza/sistem-penilaian-kompetensi-v2

!pip install batchbald_redux

import sys
sys.path.append('/content/sistem-penilaian-kompetensi-v2/Penilaian Kompetensi Inggris')

Cloning into 'sistem-penilaian-kompetensi-v2'...
remote: Enumerating objects: 19203, done.
remote: Counting objects: 100% (1075/1075), done.
remote: Compressing objects: 100% (372/372), done.
remote: Total 19203 (delta 675), reused 1075 (delta 675), pack-reused 18128
Receiving objects: 100% (19203/19203), 163.77 MiB | 12.81 MiB/s, done.
Resolving deltas: 100% (2472/2472), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nv

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving dataset.json to dataset.json


In [ ]:
import json
for key in uploaded.keys():
  raw_data = json.loads(uploaded[key])['data']

# Evaluation Setup

In [ ]:
from src.model import *
import torch

In [ ]:
device = torch.device('cuda')

In [ ]:
# Set seed
torch.manual_seed(0)

# Active Learning Evaluation

Data split: 7 pool sets / 2 eval sets (1 set = 4 data points)<br>
Base model: sentence-transformers/all-mpnet-base-v2<br>

Acquisition function: Random<br>
Acquisition size: 10<br>
Acquisition step: 2<br>

In [ ]:
pool_data = raw_data[:7]
eval_data = raw_data[7:]

transcripts = [transcript for d in pool_data for transcript in d['transcripts']]
competence_sets = [d['competence_levels'] for d in pool_data for _ in range(len(d['transcripts']))]
labels = [label for d in pool_data for label in d['labels']]
pool_dataset = TCDataset(transcripts, competence_sets, labels)

transcripts = [transcript for d in eval_data for transcript in d['transcripts']]
competence_sets = [d['competence_levels'] for d in eval_data for _ in range(len(d['transcripts']))]
labels = [label for d in eval_data for label in d['labels']]
eval_dataset = TCDataset(transcripts, competence_sets, labels)

In [ ]:
model = CompetenceModel.load('sentence-transformers/all-mpnet-base-v2', 'biencoder', state_dict_path=None, device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# Model prediction on eval dataset (initial)

with torch.no_grad():
    scores = model(eval_dataset.transcripts, eval_dataset.competence_sets)
    acc = torch.sum(torch.argmax(scores, dim=1) == torch.tensor(eval_dataset.label_indices, device=device)).item() / scores.size(0)

print('Predict probabilities:')
print(scores)
print(f'Accuracy: {acc:.2f}')

Predict probabilities:
tensor([[0.2627, 0.1829, 0.1910, 0.3634],
        [0.2645, 0.2284, 0.2038, 0.3033],
        [0.1308, 0.1516, 0.2125, 0.5051],
        [0.1232, 0.1987, 0.2204, 0.4577],
        [0.2134, 0.2728, 0.2259, 0.2878],
        [0.1900, 0.3128, 0.2226, 0.2746],
        [0.1912, 0.2685, 0.2554, 0.2849],
        [0.2083, 0.2866, 0.2056, 0.2995]], device='cuda:0')
Accuracy: 0.38


## Acquisition Step 1

In [ ]:
batch = get_random_batch(len(pool_dataset), batch_size=10)

In [ ]:
train_dataset = torch.utils.data.Subset(pool_dataset, batch.indices)

transcripts = [t for i, t in enumerate(pool_dataset.transcripts) if i not in batch.indices]
competence_sets = [c for i, c in enumerate(pool_dataset.competence_sets) if i not in batch.indices]
labels = [l for i, l in enumerate(pool_dataset.label_indices) if i not in batch.indices]
pool_dataset = TCDataset(transcripts, competence_sets, labels)

In [ ]:
# Model training on 5 epochs

model = CompetenceModel.load('sentence-transformers/all-mpnet-base-v2', 'biencoder', state_dict_path=None, device=device)

model.fit(train_dataset, eval_dataset, epochs=5, batch_size=4, early_stop=False,
          optimizer_cls=torch.optim.Adam, optimizer_params={'lr': 1e-5})

Epoch 1 | Train Loss = 1.4063 | Train Acc = 0.3000 | Val Loss = 1.3086 | Val Acc = 0.3750
Epoch 2 | Train Loss = 1.2938 | Train Acc = 0.4000 | Val Loss = 1.3094 | Val Acc = 0.3750
Epoch 3 | Train Loss = 1.2306 | Train Acc = 0.5000 | Val Loss = 1.3156 | Val Acc = 0.3750
Epoch 4 | Train Loss = 1.0841 | Train Acc = 0.7000 | Val Loss = 6.5982 | Val Acc = 0.3750
Epoch 5 | Train Loss = 0.8058 | Train Acc = 0.7000 | Val Loss = 6.5467 | Val Acc = 0.3750


In [ ]:
# Model prediction on eval dataset

with torch.no_grad():
    scores = model(eval_dataset.transcripts, eval_dataset.competence_sets)
    acc = torch.sum(torch.argmax(scores, dim=1) == torch.tensor(eval_dataset.label_indices, device=device)).item() / scores.size(0)

print('Predict probabilities:')
print(scores)
print(f'Accuracy: {acc:.2f}')

Predict probabilities:
tensor([[2.3639e-01, 5.0875e-20, 5.0875e-20, 7.6361e-01],
        [2.7915e-01, 1.6774e-01, 1.2731e-01, 4.2580e-01],
        [1.8615e-19, 1.8615e-19, 1.8615e-19, 1.0000e+00],
        [3.7236e-20, 3.7236e-20, 4.4582e-02, 9.5542e-01],
        [8.1136e-02, 3.7490e-01, 8.1110e-02, 4.6286e-01],
        [1.7506e-20, 5.2578e-01, 1.0890e-02, 4.6333e-01],
        [2.5955e-02, 3.8376e-01, 1.4560e-01, 4.4469e-01],
        [1.2602e-01, 3.6447e-01, 9.9374e-02, 4.1014e-01]], device='cuda:0')
Accuracy: 0.38


## Acquisition Step 2

In [ ]:
batch = get_random_batch(len(pool_dataset), batch_size=10)

In [ ]:
train_dataset = torch.utils.data.ConcatDataset([train_dataset, torch.utils.data.Subset(pool_dataset, batch.indices)])

transcripts = [t for i, t in enumerate(pool_dataset.transcripts) if i not in batch.indices]
competence_sets = [c for i, c in enumerate(pool_dataset.competence_sets) if i not in batch.indices]
labels = [l for i, l in enumerate(pool_dataset.label_indices) if i not in batch.indices]
pool_dataset = TCDataset(transcripts, competence_sets, labels)

In [ ]:
# Model training on 5 epochs

model = CompetenceModel.load('sentence-transformers/all-mpnet-base-v2', 'biencoder', state_dict_path=None, device=device)

model.fit(train_dataset, eval_dataset, epochs=5, batch_size=4, early_stop=False,
          optimizer_cls=torch.optim.Adam, optimizer_params={'lr': 1e-5})

Epoch 1 | Train Loss = 1.3908 | Train Acc = 0.3000 | Val Loss = 1.2912 | Val Acc = 0.3750
Epoch 2 | Train Loss = 1.2874 | Train Acc = 0.3500 | Val Loss = 1.2766 | Val Acc = 0.3750
Epoch 3 | Train Loss = 1.1632 | Train Acc = 0.5000 | Val Loss = 1.3163 | Val Acc = 0.3750
Epoch 4 | Train Loss = 0.9122 | Train Acc = 0.6500 | Val Loss = 6.2903 | Val Acc = 0.5000
Epoch 5 | Train Loss = 4.9016 | Train Acc = 0.6000 | Val Loss = 11.2975 | Val Acc = 0.5000


In [ ]:
# Model prediction on eval dataset after training

with torch.no_grad():
    scores = model(eval_dataset.transcripts, eval_dataset.competence_sets)
    acc = torch.sum(torch.argmax(scores, dim=1) == torch.tensor(eval_dataset.label_indices, device=device)).item() / scores.size(0)

print('Predict probabilities:')
print(scores)
print(f'Accuracy: {acc:.2f}')

Predict probabilities:
tensor([[9.0845e-01, 2.0719e-19, 2.0719e-19, 9.1545e-02],
        [1.0000e+00, 4.6839e-19, 4.6839e-19, 4.6839e-19],
        [2.5000e-01, 2.5000e-01, 2.5000e-01, 2.5000e-01],
        [2.5000e-01, 2.5000e-01, 2.5000e-01, 2.5000e-01],
        [2.9851e-01, 2.8603e-01, 1.7365e-01, 2.4182e-01],
        [7.8695e-20, 6.1199e-01, 7.8695e-20, 3.8801e-01],
        [2.7250e-19, 2.7250e-19, 2.7250e-19, 1.0000e+00],
        [1.6761e-01, 3.3184e-01, 7.7342e-02, 4.2321e-01]], device='cuda:0')
Accuracy: 0.50
